<a href="https://colab.research.google.com/github/danilovabg/SkillFactory/blob/master/hw_ml7_DanilovaA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import  GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [14]:
data = pd.read_csv('_train_sem09 (1).csv')
data.tail(3)

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
3748,0,0.200000,0.520564,0.0,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.0,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0
3750,0,0.133333,0.533952,0.0,0.0,0.347966,0.757971,0.230667,0.272652,0.854116,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)

# Logistic regression

In [16]:
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)
log_pred = logreg.predict(X_test)
print(f'F1 at test {metrics.f1_score(y_test, log_pred):.2f}')
print(f'F1 at train {metrics.f1_score(y_train, logreg.predict(X_train)):.2f}')


F1 at test 0.78
F1 at train 0.89


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Random Forest

In [17]:
forest = RandomForestClassifier(random_state=42)
forest.fit(X_train, y_train)
forest_pred = forest.predict(X_test)
print(f'F1 at test {metrics.f1_score(y_test, forest_pred):.2f}')
print(f'F1 at train {metrics.f1_score(y_train, forest.predict(X_train)):.2f}')

F1 at test 0.80
F1 at train 1.00


Метрика на трейне равна 1, при этом метрика на тесте 0,8
видно переобучение теме не менее метрика случайного леса на тесте выше метрики логистической регрессии

# GridSearch

## Logistic regression with GridSearch

In [26]:
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации

              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=1, max_iter=50),
    param_grid=param_grid,
    cv=5,
    n_jobs = -1
)
%time grid_search_1.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_1.score(X_test, y_test)))
y_test_pred = grid_search_1.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

CPU times: user 10.1 s, sys: 1.06 s, total: 11.2 s
Wall time: 8min 28s
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [29]:
y_train_pred = grid_search_1.predict(X_train)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

f1_score на тестовом наборе: 0.79
f1_score на тренировочном наборе: 0.83
Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}


# Random Forest with GridSearch

In [20]:
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 5, dtype=int))
              }

grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    n_jobs = -1
)
%time grid_search_forest.fit(X_train, y_train)
y_train_pred = grid_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_forest.score(X_test, y_test)))
y_test_pred = grid_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

CPU times: user 4.15 s, sys: 365 ms, total: 4.51 s
Wall time: 2min 22s
f1_score на обучающем наборе: 0.94
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 140}


In [30]:
y_train_pred = grid_search_forest.predict(X_train)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

f1_score на тестовом наборе: 0.80
f1_score на тренировочном наборе: 0.94
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 140}


После поиска гиперпараметров по сетке видно что на модели логистической регрессии немного подросла метрика, но поиск параметров модели случайного леса не дал прироста метрики, для дальнейшего поиска нужно изменить параметры сетки

# RandomizedSearch

## RandomizedSearch with Logistic regression

In [33]:
from sklearn.model_selection import RandomizedSearchCV

#np.linspace(start(от), stop(до), num=50(количество),dtype- тип данных)
param_distributions = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))}

random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50),
    param_distributions=param_distributions,
    cv=5,
    n_iter = 10,
    n_jobs = -1
)
%time random_search.fit(X_train, y_train)
y_train_pred = random_search.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, y_test)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: user 1.87 s, sys: 344 ms, total: 2.22 s
Wall time: 1min 11s
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78
f1_score на тренировочном наборе: 0.82
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


У линейной модели опять не видно улучшения результатов

## RandomizedSearch with RandomForest

In [35]:
param_grid = {'n_estimators': list(range(100, 200, 30)),
              'min_samples_leaf': [5, 7],
              'max_depth' : [10, 15, 20, 25]
              }

grid_search_tree = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42),
    param_grid = param_grid,
    cv=5,
    n_jobs = -1
)
%time grid_search_tree.fit(X_train, y_train)
y_train_pred = grid_search_tree.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_tree.score(X_test, y_test)))
y_test_pred = grid_search_tree.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_tree.best_params_))

CPU times: user 5.45 s, sys: 582 ms, total: 6.03 s
Wall time: 3min 58s
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
f1_score на тренировочном наборе: 0.94
Наилучшие значения гиперпараметров: {'max_depth': 15, 'min_samples_leaf': 5, 'n_estimators': 190}


Как и в предыдущих вариантах случайный лес показывает лучшее значение метрики в сравнении с линейной регрессией

In [39]:
!pip install hyperopt==0.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.0/966.0 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.2.7
    Uninstalling hyperopt-0.2.7:
      Successfully uninstalled hyperopt-0.2.7


In [40]:
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
print("Версия Hyperopt : {}".format(hyperopt.__version__))

Версия Hyperopt : 0.2.5


In [42]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 300, 10),
       'max_depth' : hp.quniform('max_depth', 15, 40, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 7, 1)
      }

#  зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']),
              'max_depth': int(params['max_depth']),
             'min_samples_leaf': int(params['min_samples_leaf'])
              }

    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))

    return -score


trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state,
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

100%|██████████| 20/20 [01:23<00:00,  4.17s/trial, best loss: -0.9746719560573696]
Наилучшие значения гиперпараметров {'max_depth': 36.0, 'min_samples_leaf': 3.0, 'n_estimators': 210.0}
f1_score на обучающем наборе: 0.97
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
f1_score на тренировочном наборе: 0.97


Метрика опять осталась той же что и при других способах оптимизации гиперпараметров

In [44]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.8 MB/s eta 0:00:00


In [45]:
import optuna

print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.4.0


In [47]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

  %%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2023-11-16 14:18:09,694] A new study created in memory with name: RandomForestClassifier
[I 2023-11-16 14:18:16,690] Trial 0 finished with value: 0.9287454323995127 and parameters: {'n_estimators': 141, 'max_depth': 27, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9287454323995127.
[I 2023-11-16 14:18:20,939] Trial 1 finished with value: 0.8999087313659873 and parameters: {'n_estimators': 137, 'max_depth': 12, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.9287454323995127.
[I 2023-11-16 14:18:23,755] Trial 2 finished with value: 0.9333740831295844 and parameters: {'n_estimators': 106, 'max_depth': 12, 'min_samples_leaf': 4}. Best is trial 2 with value: 0.9333740831295844.
[I 2023-11-16 14:18:32,507] Trial 3 finished with value: 0.9009118541033436 and parameters: {'n_estimators': 196, 'max_depth': 15, 'min_samples_leaf': 9}. Best is trial 2 with value: 0.9333740831295844.
[I 2023-11-16 14:18:35,396] Trial 4 finished with value: 0.9296803652968038 and parameters: {'n_

In [48]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
f1_score на тренировочном наборе: 0.99


Я провела еще несколько опытов с расширенными параметрами и бОльшим количеством "точек" поиска, но результаты остались такими же

Лучшее значение метрики на тесте 0.8